Setup

In [1]:
#setup
import os
import sys
# Define the path to the project root directory (one level above `src`)
project_root_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH"

# Add the src directory to sys.path
sys.path.append(os.path.join(project_root_path))

# Verify that src is in sys.path
print(sys.path)
import os
import argparse
from src.utils.dataset_utils import create_directory
from src.visualizers.Open3dVisualizer import Open3DVisualizer
from src.utils.json_utils import load_cuboid_data
from src.utils.plotting_utils import load_mesh_data, load_pointcloud_data
from src.scripts.preprocessing_data.data_to_6views_utils import process_file
import numpy as np
from PIL import Image
import open3d as o3d
import matplotlib.pyplot as plt
import argparse
import os

import cv2
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm


['/opt/anaconda3/envs/myenv/lib/python311.zip', '/opt/anaconda3/envs/myenv/lib/python3.11', '/opt/anaconda3/envs/myenv/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/myenv/lib/python3.11/site-packages', '/Users/cmazzoleni/Documents/GitHub/Hierarchical-Localization/Hierarchical-Localization/Hierarchical-Localization', '/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH']


# Load mesh


In [2]:
visualizer = Open3DVisualizer()

mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
visualizer.vis.add_geometry(mesh)

front_view_direction = [0, 0, -1] 

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


 # Capture rgb 

In [3]:
front_view_direction = [0, 0, -1]  # Front view direction (camera looking towards -z)

# Save the front view RGB image
front_rgb_mesh_filename = "front_rgb_mesh_image.png"
visualizer.capture_front_picture(front_rgb_mesh_filename)

rgb = np.array(Image.open(front_rgb_mesh_filename))



Front view screenshot saved as front_rgb_mesh_image.png


# Capture depth

In [8]:
# Save the front view depth image
front_depth_mesh_filename = "front_depth_mesh_image.png"
visualizer.capture_front_depth(front_depth_mesh_filename, distance=1.0, depth_scale=1000.0)

# The front_depth_filename will store the normalized depth image
#find min and max values
depth_image = np.array(Image.open(front_depth_mesh_filename))
min_depth = np.min(depth_image)
max_depth = np.max(depth_image)
print("min_depth: ", min_depth)
print("max_depth: ", max_depth)


Front view screenshot saved as front_depth_mesh_image.png
min_depth:  0
max_depth:  1096


# Hypersim processing Tonemap


In [5]:
def tone_map(rgb):
    gamma = 1.0 / 2.2  # standard gamma correction exponent
    inv_gamma = 1.0 / gamma
    percentile = 90  # we want this percentile brightness value in the unmodified image...
    brightness_nth_percentile_desired = 0.8  # ...to be this bright after scaling

    # Generate a synthetic entity_id_map where all pixels are valid
    entity_id_map = np.ones(rgb.shape[:2], dtype=np.int32)

    valid_mask = entity_id_map != -1  # In this case, all pixels are valid
    brightness = (
        0.3 * rgb[:, :, 0] + 0.59 * rgb[:, :, 1] + 0.11 * rgb[:, :, 2]
    )
    brightness_valid = brightness[valid_mask]

    eps = 0.0001
    brightness_nth_percentile_current = np.percentile(brightness_valid, percentile)

    if brightness_nth_percentile_current < eps:
        scale = 0.0
    else:
        scale = (
            np.power(brightness_nth_percentile_desired, inv_gamma)
            / brightness_nth_percentile_current
        )

    # Apply tone mapping
    rgb_color_tm = np.power(np.maximum(scale * rgb, 0), gamma)
    rgb_color_tm = np.clip(rgb_color_tm, 0, 1)

    return rgb_color_tm



In [6]:
#apply tone map to rgb image

rgb_color_tm = tone_map(rgb)
rgb_int = (rgb_color_tm * 255).astype(np.uint8)  # [H, W, RGB]
cv2.imwrite("front_rgb_mesh_image_tonemapped.png", cv2.cvtColor(rgb_int, cv2.COLOR_RGB2BGR))

True

# Hypersim processing dist to depth

In [ ]:
# According to https://github.com/apple/ml-hypersim/issues/9
def dist_2_depth(width, height, flt_focal, distance):
    img_plane_x = (
        np.linspace((-0.5 * width) + 0.5, (0.5 * width) - 0.5, width)
        .reshape(1, width)
        .repeat(height, 0)
        .astype(np.float32)[:, :, None]
    )
    img_plane_y = (
        np.linspace((-0.5 * height) + 0.5, (0.5 * height) - 0.5, height)
        .reshape(height, 1)
        .repeat(width, 1)
        .astype(np.float32)[:, :, None]
    )
    img_plane_z = np.full([height, width, 1], flt_focal, np.float32)
    img_plane = np.concatenate([img_plane_x, img_plane_y, img_plane_z], 2)

    depth = distance / np.linalg.norm(img_plane, 2, 2) * flt_focal
    return depth

In [20]:
plane_depth = depth_image

print(plane_depth.shape)
print(plane_depth.dtype)
print(plane_depth.min())
print(plane_depth.max())
cv2.imwrite("front_depth_mesh_image_cv2.png", plane_depth)


(800, 800)
uint16
0
1096


True

In [21]:
import open3d as o3d
import numpy as np

def reconstruct_from_depth(depth_filename, width, height, fx, fy, cx, cy, output_pointcloud_filename):
    # Load the saved raw depth image (not normalized)
    depth_image = o3d.io.read_image(depth_filename)
    depth_np = np.asarray(depth_image)

    # Print the data type and shape of the depth image
    print(f"Depth image type: {depth_np.dtype}")
    print(f"Depth image shape: {depth_np.shape}")

    # Create an intrinsic matrix for the camera
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

    # Convert the depth image to a point cloud
    pcd = o3d.geometry.PointCloud.create_from_depth_image(
        depth_image, intrinsic, extrinsic=np.identity(4)
    )

    # Save the reconstructed point cloud to a file
    o3d.io.write_point_cloud(output_pointcloud_filename, pcd)
    print(f"Reconstructed point cloud saved to {output_pointcloud_filename}")

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

# Example parameters
depth_filename = "front_depth_mesh_image_cv2.png"  # The raw depth image file
output_pointcloud_filename = "reconstructed_pointcloud.ply"

# Intrinsic parameters (adjust based on your setup)
width = 800  # Image width
height = 800  # Image height
fx = 692.82  # Focal length in x direction
fy = 692.82  # Focal length in y direction
cx = width / 2.0  # Principal point in x direction
cy = height / 2.0  # Principal point in y direction

# Call the reconstruction function
reconstruct_from_depth(depth_filename, width, height, fx, fy, cx, cy, output_pointcloud_filename)


Depth image type: uint16
Depth image shape: (800, 800)
Reconstructed point cloud saved to reconstructed_pointcloud.ply


: 